# Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Lab Exam:
# SOLVE = 1
# Step 1 : Reading & Analyzing Cleaned Comments

In [ ]:
import pandas as pd


file_path = "/content/drive/MyDrive/477_lab_exam/cleaned_comments.csv"
df = pd.read_csv(file_path)


print("Dataset Shape (rows, cols):", df.shape)
print("\nFirst 5 rows:")
display(df.head())

print("\nDataset Info:")
df.info()





# Step 2 : Handle rows consistently
 Drop duplicate rows (if any)

In [ ]:
df = df.drop_duplicates()

# Handle missing values → drop rows with all NaN, fill others with empty string
df = df.dropna(how="all")
df = df.fillna("")

print("\nAfter cleaning:")
print("Shape:", df.shape)



# Step 3: Basic Analysis
- Number of comments
- Average length of comments
- Show 5 longest comments

In [ ]:
print("\nTotal number of comments:", len(df))


df["comment_length"] = df.iloc[:,0].astype(str).apply(len)
print("Average comment length:", df["comment_length"].mean())


print("\nTop 5 Longest Comments:")
display(df.sort_values("comment_length", ascending=False).head())


print("\nDescriptive Stats:")
display(df["comment_length"].describe())


# SOLVE = 2

In [ ]:
# Cell 2: (Optional) Install packages for extra text processing/visuals.
# If you don't need lemmatization or wordcloud, you can skip this cell.
!pip install -q nltk



In [ ]:
# Cell 3: imports and helper functions
import os
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

# basic text cleaning function
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    # remove urls, mentions, hashtags
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    text = re.sub(r'@\w+|#\w+', ' ', text)
    # keep alphanumerics and simple punctuation, replace others with space
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
# Cell 4: Load CSV and auto-detect likely text column
file_path = "/content/drive/MyDrive/477_lab_exam/cleaned_comments.csv"
df = pd.read_csv(file_path)

print("Original shape:", df.shape)
display(df.head(5))

# try to auto-detect comment/text column by common names
possible_names = ['comment', 'comments', 'text', 'cleaned_comment', 'cleaned_comments', 'message', 'content']
text_col = None
for name in possible_names:
    if name in df.columns.str.lower():
        # keep matching original case
        text_col = df.columns[df.columns.str.lower() == name][0]
        break

# If not found, pick the first column (common in student datasets)
if text_col is None:
    text_col = df.columns[0]
    print(f"Did not find a standard text column. Using first column: '{text_col}'")
else:
    print(f"Using detected text column: '{text_col}'")

# Ensure it's string
df[text_col] = df[text_col].astype(str)


In [ ]:
# Cell 5: Drop rows that are totally empty, fill partial NaNs, and drop exact duplicates
# Drop rows where the chosen text column is empty / whitespace only
df[text_col] = df[text_col].apply(lambda x: x.strip() if isinstance(x, str) else x)
initial_shape = df.shape

# drop rows where text is empty
df = df[~df[text_col].isin(['', 'nan', 'None', None])]

# drop exact duplicate rows (all columns identical)
df = df.drop_duplicates()

print("Initial shape:", initial_shape)
print("After dropping empty/duplicate rows:", df.shape)

# preview
display(df.head(8))


In [ ]:
# Cell 6: Apply clean_text to create a preprocessed column for vectorization
df['cleaned_text'] = df[text_col].apply(clean_text)

# If any rows became empty after cleaning (rare), drop them
df = df[df['cleaned_text'].str.strip() != '']

print("After cleaning, sample cleaned_text:")
display(df[['cleaned_text']].head(8))


In [ ]:
# Cell 8: TF-IDF vectorization
# If you ran lemmatization cell, uncomment the following line; otherwise use cleaned_text:
# use_col = 'cleaned_text_lem'

# If you didn't run lemmatize, set:
use_col = 'cleaned_text' if 'cleaned_text_lem' not in df.columns else 'cleaned_text_lem'

texts = df[use_col].tolist()

# TF-IDF parameters -- tuned for exam: highlight important words and reduce noise
vectorizer = TfidfVectorizer(
    stop_words='english',   # remove common english stop words
    max_df=0.95,            # ignore very common words in corpus
    min_df=2,               # ignore words that appear in only 1 document
    max_features=15000,     # limit vocabulary size (adjust if you have tiny dataset)
    ngram_range=(1,2)       # use unigrams and bigrams (helps capture short phrases)
)
X = vectorizer.fit_transform(texts)
print("TF-IDF matrix shape:", X.shape)


In [ ]:
# Cell 9: Clustering into exactly 3 groups
n_clusters = 3
num_docs = X.shape[0]

# For very large datasets, MiniBatchKMeans is faster and memory-friendly
if num_docs > 20000:
    print("Large dataset detected — using MiniBatchKMeans for speed/memory.")
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=1000, n_init=10)
else:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

labels = kmeans.fit_predict(X)
df['Group'] = labels  # each comment gets exactly one group label: 0,1,2

print("Cluster assignment complete.")
print("Group counts:")
print(df['Group'].value_counts().sort_index())


In [ ]:
# Cell 10: Save results with Group column
output_path = "/content/drive/MyDrive/477_lab_exam/grouped_comments.csv"
df.to_csv(output_path, index=False)
print("Saved grouped file to:", output_path)


In [ ]:
# Cell 11: Inspect clusters: sample comments and top words per cluster
display(df[[text_col, use_col, 'Group']].sample(12, random_state=42))

# Show top keywords per cluster (based on cluster centers)
terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

for i in range(n_clusters):
    top_terms = [terms[ind] for ind in order_centroids[i, :15] if ind < len(terms)]
    print(f"\nTop words for Group {i} (top 15):")
    print(", ".join(top_terms))

# Show 5 example comments per group
for i in range(n_clusters):
    print(f"\n--- Examples from Group {i} ---")
    display(df[df['Group'] == i][[text_col]].head(5))


In [ ]:
# Cell 12: 2D visualization (TruncatedSVD for sparse TF-IDF)
svd = TruncatedSVD(n_components=2, random_state=42)
X2 = svd.fit_transform(X)

plt.figure(figsize=(10,6))
scatter = plt.scatter(X2[:,0], X2[:,1], c=df['Group'], alpha=0.6)
plt.title("2D projection of comments (TruncatedSVD) — colored by Group")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(scatter, ticks=[0,1,2], label='Group')
plt.show()


# SOLVE = 3

In [ ]:
# Cell 13: Prepare environment and basic checks (run after your previous clustering cells)
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Ensure required variables exist (these come from previous pipeline)
assert 'df' in globals(), "Run earlier cells first so 'df' exists."
assert 'vectorizer' in globals(), "Run TF-IDF cell first to create 'vectorizer'."
assert 'kmeans' in globals(), "Run clustering cell first to create 'kmeans'."
assert 'use_col' in globals(), "Ensure 'use_col' points to your cleaned-text column."

terms = vectorizer.get_feature_names_out()
n_clusters = kmeans.n_clusters if hasattr(kmeans, 'n_clusters') else 3

print("Ready. Number of clusters:", n_clusters)
print("TF-IDF vocabulary size:", len(terms))


In [ ]:
# Cell 14: Compute per-cluster combined score and choose the top keyword
count_vectorizer = CountVectorizer(vocabulary=terms)  # align counts to TF-IDF terms

def normalize_array(a):
    a = a.astype(float)
    a_min, a_max = a.min(), a.max()
    if a_max == a_min:
        # If all identical, return zeros (or ones) — use zeros so counts don't dominate
        return np.zeros_like(a)
    return (a - a_min) / (a_max - a_min)

group_infos = []

for i in range(n_clusters):
    # indices & texts in this cluster
    docs_idx = df[df['Group'] == i].index
    cluster_texts = df.loc[docs_idx, use_col].astype(str).tolist()
    if len(cluster_texts) == 0:
        group_infos.append({
            'group': i,
            'chosen_word': None,
            'note': 'empty_cluster',
            'top_tfidf_word': None,
            'top_count_word': None
        })
        continue

    # 1) centroid weights (TF-IDF importance for cluster)
    centroid = kmeans.cluster_centers_[i]  # shape (n_terms,)
    # 2) raw term counts in the cluster (using same vocab)
    term_count_matrix = count_vectorizer.transform(cluster_texts)  # docs x vocab
    term_counts = np.asarray(term_count_matrix.sum(axis=0)).ravel()  # shape (n_terms,)

    # normalize both signals to [0,1]
    centroid_norm = normalize_array(centroid)
    counts_norm = normalize_array(term_counts)

    # combined score (equal weight to both signals)
    combined_score = centroid_norm + counts_norm

    # pick best index
    best_idx = int(np.argmax(combined_score))
    chosen_word = terms[best_idx]

    # for transparency collect also the top pure-tfidf and top pure-count words
    top_tfidf_idx = int(np.argmax(centroid))
    top_count_idx = int(np.argmax(term_counts))
    top_tfidf_word = terms[top_tfidf_idx]
    top_count_word = terms[top_count_idx]

    group_infos.append({
        'group': i,
        'chosen_word': chosen_word,
        'chosen_score': float(combined_score[best_idx]),
        'top_tfidf_word': top_tfidf_word,
        'top_tfidf_weight': float(centroid[top_tfidf_idx]),
        'top_count_word': top_count_word,
        'top_count': int(term_counts[top_count_idx])
    })

# Create DataFrame of diagnostics
group_df = pd.DataFrame(group_infos).sort_values('group').reset_index(drop=True)
display(group_df)


In [ ]:
# Cell 15: Make the final "Group number" ↔ "Chosen keyword" table, add a nicer Label, and save
label_table = group_df[['group', 'chosen_word']].copy()
label_table.columns = ['Group', 'Chosen_Keyword']

# Make a presentable Label (Title Case) and handle missing picks
label_table['Label'] = label_table['Chosen_Keyword'].fillna('Unknown').astype(str).str.replace('_',' ').str.title()

# Final 2-column table (Group number, Chosen keyword label)
final_table = label_table[['Group', 'Label']].copy()

# Display nicely
display(final_table.style.set_caption("Group → Representative Keyword").format({'Group': '{:.0f}'}))

# Save to Drive
output_labels_path = "/content/drive/MyDrive/477_lab_exam/group_labels.csv"
final_table.to_csv(output_labels_path, index=False)
print("Saved group labels to:", output_labels_path)


In [ ]:
# Cell 16: Map chosen labels back onto df and show quick examples per labeled group
label_map = dict(zip(final_table['Group'], final_table['Label']))
df['Group_Label'] = df['Group'].map(label_map)

print("Counts per labeled group:")
display(df['Group_Label'].value_counts().rename_axis('Label').reset_index(name='Count'))

# Show up to 10 sample comments per labeled group with the label visible
for grp in sorted(df['Group'].unique()):
    lbl = label_map.get(grp, f"Group {grp}")
    print(f"\n=== Samples for {lbl} (Group {grp}) ===")
    display(df[df['Group'] == grp][[use_col]].head(10))


In [ ]:
# Cell 17: Install wordcloud package (if not already installed)
!pip install -q wordcloud

In [ ]:
# Cell 18: Plot top 10 TF-IDF words for each group
import matplotlib.pyplot as plt

terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

for i in range(n_clusters):
    top_indices = order_centroids[i, :10]
    top_terms = [terms[ind] for ind in top_indices]
    top_weights = [kmeans.cluster_centers_[i, ind] for ind in top_indices]

    plt.figure(figsize=(8,4))
    plt.barh(top_terms[::-1], top_weights[::-1], color="skyblue")
    plt.title(f"Group {i} — Top 10 Words (TF-IDF weight)")
    plt.xlabel("TF-IDF Weight")
    plt.ylabel("Words")
    plt.show()


In [ ]:
# Cell 19: WordCloud visualization for each group
from wordcloud import WordCloud

for i in range(n_clusters):
    cluster_texts = df[df['Group'] == i][use_col].astype(str)
    combined_text = " ".join(cluster_texts)

    if combined_text.strip() == "":
        print(f"Group {i} is empty, skipping wordcloud.")
        continue

    wordcloud = WordCloud(
        width=800, height=400,
        background_color="white",
        colormap="viridis",
        max_words=100
    ).generate(combined_text)

    plt.figure(figsize=(10,5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"WordCloud for Group {i}")
    plt.show()


# SOLVE = 4

In [ ]:
# Install NLTK and required packages
!pip install -q nltk

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK for sentiment analysis
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer


In [ ]:
# Load your cleaned comments CSV (or the labeled one from Q3)
file_path = "/content/drive/MyDrive/477_lab_exam/grouped_comments.csv"
df = pd.read_csv(file_path)

# Check columns
print("Dataset columns:", df.columns)
print("First 5 rows:")
display(df.head())

# We'll use 'cleaned_text' column for sentiment
use_col = 'cleaned_text'


In [ ]:
# Initialize VADER Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Example: analyze a single comment
example_text = df[use_col].iloc[0]
sia.polarity_scores(example_text)


In [ ]:
# Function to map VADER compound score to sentiment label
def get_sentiment_label(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis
df['Sentiment'] = df[use_col].apply(get_sentiment_label)

# Quick stats
print("Sentiment distribution:")
display(df['Sentiment'].value_counts())


In [ ]:
# Plot sentiment distribution
plt.figure(figsize=(7,5))
sns.countplot(x='Sentiment', data=df, order=['Positive', 'Neutral', 'Negative'], palette='pastel')
plt.title("Sentiment Distribution of Comments")
plt.ylabel("Number of Comments")
plt.xlabel("Sentiment")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# How sentiments distribute across groups
sentiment_group = df.groupby(['Group', 'Sentiment']).size().reset_index(name='Count')

plt.figure(figsize=(10,5))
sns.barplot(x='Group', y='Count', hue='Sentiment', data=sentiment_group, palette='Set2')
plt.title("Sentiment Distribution Across Groups")
plt.ylabel("Number of Comments")
plt.xlabel("Group")
plt.legend(title="Sentiment")
plt.show()


In [ ]:
# Save final dataset with Group, Label, Sentiment
output_path_sentiment = "/content/drive/MyDrive/477_lab_exam/grouped_comments_sentiment.csv"
df.to_csv(output_path_sentiment, index=False)
print(f"✅ Saved dataset with sentiment labels: {output_path_sentiment}")


# SOLVE = 5

In [ ]:
# Question 5: Visualize Group Distribution
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


file_path = "/content/drive/MyDrive/477_lab_exam/grouped_comments_sentiment.csv"
df = pd.read_csv(file_path)


print("Groups present in dataset:", df['Group'].unique())




In [ ]:

group_counts = df['Group'].value_counts().sort_index()
print("Number of comments per group:")
display(group_counts)



In [ ]:

plt.figure(figsize=(8,5))
sns.barplot(x=group_counts.index, y=group_counts.values, palette='pastel')


for i, count in enumerate(group_counts.values):
    plt.text(i, count + max(group_counts.values)*0.01, str(count), ha='center', va='bottom', fontsize=12)

plt.title("Distribution of Comments Across Groups", fontsize=14)
plt.xlabel("Group", fontsize=12)
plt.ylabel("Number of Comments", fontsize=12)
plt.xticks(ticks=[0,1,2], labels=[f"Group {i}" for i in group_counts.index])
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()



In [ ]:
output_img_path = "/content/drive/MyDrive/477_lab_exam/group_distribution.png"
plt.savefig(output_img_path, dpi=300)
plt.show()

print(f"✅ Saved group distribution chart as: {output_img_path}")



In [ ]:
plt.figure(figsize=(6,6))
plt.pie(group_counts.values, labels=[f"Group {i}" for i in group_counts.index],
        autopct='%1.1f%%', colors=sns.color_palette('pastel'), startangle=90)
plt.title("Proportion of Comments in Each Group")
plt.tight_layout()


output_pie_path = "/content/drive/MyDrive/477_lab_exam/group_distribution_pie.png"
plt.savefig(output_pie_path, dpi=300)
plt.show()

print(f"✅ Saved pie chart as: {output_pie_path}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


file_path = "/content/drive/MyDrive/477_lab_exam/grouped_comments_sentiment.csv"
df = pd.read_csv(file_path)

total_comments = len(df)


group_counts = df['Group'].value_counts().sort_index()


fail_counts = total_comments - group_counts
print("Number of comments that FAIL to each group:")
display(fail_counts)


plt.figure(figsize=(8,5))
sns.barplot(x=fail_counts.index, y=fail_counts.values, palette='pastel')


for i, count in enumerate(fail_counts.values):
    plt.text(i, count + max(fail_counts.values)*0.01, str(count),
             ha='center', va='bottom', fontsize=12)

plt.title("Number of Comments Failing to Each Group", fontsize=14)
plt.xlabel("Group", fontsize=12)
plt.ylabel("Number of Comments", fontsize=12)
plt.xticks(ticks=[0,1,2], labels=[f"Group {i}" for i in fail_counts.index])
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()


bar_chart_path = "/content/drive/MyDrive/477_lab_exam/fail_to_group_bar.png"
plt.savefig(bar_chart_path, dpi=300)
plt.show()
print(f"✅ Bar chart saved at: {bar_chart_path}")




In [ ]:
plt.figure(figsize=(6,6))
plt.pie(fail_counts.values, labels=[f"Group {i}" for i in fail_counts.index],
        autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
plt.title("Proportion of Comments Failing to Each Group", fontsize=14)
plt.tight_layout()

# Save pie chart
pie_chart_path = "/content/drive/MyDrive/477_lab_exam/fail_to_group_pie.png"
plt.savefig(pie_chart_path, dpi=300)
plt.show()
print(f"✅ Pie chart saved at: {pie_chart_path}")